<a href="https://colab.research.google.com/github/Soumo31428/RandomForestClassification/blob/main/Random_Forest_Classification_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv('/content/Travel.csv')
df.head()

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3,3.0,Deluxe,3.0,Single,1.0,1,2,1,0.0,Manager,20993.0
1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,3,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,2.0,Manager,20130.0
2,200002,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,3,4.0,Basic,3.0,Single,7.0,1,3,0,0.0,Executive,17090.0
3,200003,0,33.0,Company Invited,1,9.0,Salaried,Female,2,3.0,Basic,3.0,Divorced,2.0,1,5,1,1.0,Executive,17909.0
4,200004,0,NaN,Self Enquiry,1,8.0,Small Business,Male,2,3.0,Basic,4.0,Divorced,1.0,0,5,1,0.0,Executive,18468.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4888 entries, 0 to 4887
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CustomerID                4888 non-null   int64  
 1   ProdTaken                 4888 non-null   int64  
 2   Age                       4662 non-null   float64
 3   TypeofContact             4863 non-null   object 
 4   CityTier                  4888 non-null   int64  
 5   DurationOfPitch           4637 non-null   float64
 6   Occupation                4888 non-null   object 
 7   Gender                    4888 non-null   object 
 8   NumberOfPersonVisiting    4888 non-null   int64  
 9   NumberOfFollowups         4843 non-null   float64
 10  ProductPitched            4888 non-null   object 
 11  PreferredPropertyStar     4862 non-null   float64
 12  MaritalStatus             4888 non-null   object 
 13  NumberOfTrips             4748 non-null   float64
 14  Passport

## Data Cleaning
### Handling Missing Values

1. Handling Missing values
2. Handling Duplicates
3. Check Data Type
4. Understand the dataset


In [7]:
df.isnull().sum()

,0
CustomerID,0
ProdTaken,0
Age,226
TypeofContact,25
CityTier,0
DurationOfPitch,251
Occupation,0
Gender,0
NumberOfPersonVisiting,0
NumberOfFollowups,45


In [8]:
## Check all the categories
df['Gender'].value_counts(), df['MaritalStatus'].value_counts(), df['TypeofContact'].value_counts()

(Gender
 Male       2916
 Female     1817
 Fe Male     155
 Name: count, dtype: int64,
 MaritalStatus
 Married      2340
 Divorced      950
 Single        916
 Unmarried     682
 Name: count, dtype: int64,
 TypeofContact
 Self Enquiry       3444
 Company Invited    1419
 Name: count, dtype: int64)

In [9]:
df['Gender'] = df['Gender'].replace('Fe Male', 'Female')
df['MaritalStatus'] = df['MaritalStatus'].replace('Single', 'Unmarried' )


In [10]:
df.head()

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3,3.0,Deluxe,3.0,Unmarried,1.0,1,2,1,0.0,Manager,20993.0
1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,3,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,2.0,Manager,20130.0
2,200002,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,3,4.0,Basic,3.0,Unmarried,7.0,1,3,0,0.0,Executive,17090.0
3,200003,0,33.0,Company Invited,1,9.0,Salaried,Female,2,3.0,Basic,3.0,Divorced,2.0,1,5,1,1.0,Executive,17909.0
4,200004,0,NaN,Self Enquiry,1,8.0,Small Business,Male,2,3.0,Basic,4.0,Divorced,1.0,0,5,1,0.0,Executive,18468.0


In [11]:
## Check Missing Values
## These are the features with nan value.
features_with_na = [features for features in df.columns if df[features].isnull().sum()>=1]
for feature in features_with_na:
  print(feature, np.round(df[feature].isnull().mean()*100,5), '% missing values')

Age 4.62357 % missing values
TypeofContact 0.51146 % missing values
DurationOfPitch 5.13502 % missing values
NumberOfFollowups 0.92062 % missing values
PreferredPropertyStar 0.53191 % missing values
NumberOfTrips 2.86416 % missing values
NumberOfChildrenVisiting 1.35025 % missing values
MonthlyIncome 4.76678 % missing values


In [12]:
## Statistcs on neumerical columns (NuLL cols)
df[features_with_na].select_dtypes(exclude='object').describe()

,Age,DurationOfPitch,NumberOfFollowups,PreferredPropertyStar,NumberOfTrips,NumberOfChildrenVisiting,MonthlyIncome
count,4662.000000,4637.000000,4843.000000,4862.000000,4748.000000,4822.000000,4655.000000
mean,37.622265,15.490835,3.708445,3.581037,3.236521,1.187267,23619.853491
std,9.316387,8.519643,1.002509,0.798009,1.849019,0.857861,5380.698361
min,18.000000,5.000000,1.000000,3.000000,1.000000,0.000000,1000.000000
25%,31.000000,9.000000,3.000000,3.000000,2.000000,1.000000,20346.000000
50%,36.000000,13.000000,4.000000,3.000000,3.000000,1.000000,22347.000000
75%,44.000000,20.000000,4.000000,4.000000,4.000000,2.000000,25571.000000
max,61.000000,127.000000,6.000000,5.000000,22.000000,3.000000,98678.000000


### Imputing Null Values
1. Impute Median for Age column
2. Impute Mode for Type of Contract
3. Impute Median for Duration of Pitch
4. Impute Mode for NumberofFollowup as it is Discrete feature
5. Impute Mode for PreferredPropertyStar
6. Impute Median for NumberofTrips
7. Impute Mode for NumberOfChildrenVisiting
8. Impute Median for MonthlyIncome

In [13]:
# Age
df.Age.fillna(df.Age.median(), inplace = True)

# TypeofContract
df.TypeofContact.fillna(df.TypeofContact.mode()[0], inplace = True)

# DurationOfFollowups
df.NumberOfFollowups.fillna(df.NumberOfFollowups.mode()[0], inplace = True)

# PreferredPropertyStar
df.PreferredPropertyStar.fillna(df.PreferredPropertyStar.mode()[0], inplace = True)

# NumberOfTrips
df.NumberOfTrips.fillna(df.NumberOfTrips.median(), inplace = True)

# NumberOfChildrenVisiting
df.NumberOfChildrenVisiting.fillna(df.NumberOfChildrenVisiting.median(), inplace = True)

# MonthlyIncome
df.MonthlyIncome.fillna(df.MonthlyIncome.median(), inplace = True)

df.isnull().sum()

,0
CustomerID,0
ProdTaken,0
Age,0
TypeofContact,0
CityTier,0
DurationOfPitch,251
Occupation,0
Gender,0
NumberOfPersonVisiting,0
NumberOfFollowups,0


In [14]:
df.drop('CustomerID', inplace = True, axis = 1)

## Feature Engineering
### Feaature Extraction

In [15]:
df.head()

,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3,3.0,Deluxe,3.0,Unmarried,1.0,1,2,1,0.0,Manager,20993.0
1,0,49.0,Company Invited,1,14.0,Salaried,Male,3,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,2.0,Manager,20130.0
2,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,3,4.0,Basic,3.0,Unmarried,7.0,1,3,0,0.0,Executive,17090.0
3,0,33.0,Company Invited,1,9.0,Salaried,Female,2,3.0,Basic,3.0,Divorced,2.0,1,5,1,1.0,Executive,17909.0
4,0,36.0,Self Enquiry,1,8.0,Small Business,Male,2,3.0,Basic,4.0,Divorced,1.0,0,5,1,0.0,Executive,18468.0


In [16]:
df['TotalVisiting'] = df.NumberOfPersonVisiting + df.NumberOfChildrenVisiting
df.drop(['NumberOfPersonVisiting', 'NumberOfChildrenVisiting'], axis = 1, inplace = True)
df.head()

,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,Designation,MonthlyIncome,TotalVisiting
0,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3.0,Deluxe,3.0,Unmarried,1.0,1,2,1,Manager,20993.0,3.0
1,0,49.0,Company Invited,1,14.0,Salaried,Male,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,Manager,20130.0,5.0
2,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,4.0,Basic,3.0,Unmarried,7.0,1,3,0,Executive,17090.0,3.0
3,0,33.0,Company Invited,1,9.0,Salaried,Female,3.0,Basic,3.0,Divorced,2.0,1,5,1,Executive,17909.0,3.0
4,0,36.0,Self Enquiry,1,8.0,Small Business,Male,3.0,Basic,4.0,Divorced,1.0,0,5,1,Executive,18468.0,2.0


In [17]:
## Get all the numeric feature
numeric_features = [feature for feature in df.columns if df[feature].dtype != 'O']
print('Number of Numerical features',len(numeric_features))

## get all the categorical feature
categorical_features = [feature for feature in df.columns if df[feature].dtype == 'O']
print('Number of categorical features',len(categorical_features))

## Discrete Features
discrete_features = [feature for feature in numeric_features if len(df[feature].unique()) <= 25]
print('Number of discrete features',len(discrete_features))

## Continuous Features
continuous_features = [feature for feature in numeric_features if feature not in discrete_features]
print('Number of continuous features',len(continuous_features))

Number of Numerical features 12
Number of categorical features 6
Number of discrete features 9
Number of continuous features 3


## train test split and Model Training

In [18]:
from sklearn.model_selection import train_test_split
x = df.drop(['ProdTaken'], axis = 1)
y = df['ProdTaken']
xtrain, xtest, ytrain, ytest = train_test_split(x,y, random_state =10, test_size = 0.2)
xtrain.shape, xtest.shape

((3910, 17), (978, 17))

In [19]:
## Create column transformer with 3 types of transformers
cat_features = x.select_dtypes(include = 'object').columns
num_features = x.select_dtypes(exclude = 'object').columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ('OneHotEncoder', oh_transformer, cat_features),
        ('StadardScaler', numeric_transformer, num_features)
    ]
)

In [20]:
preprocessor

ColumnTransformer(transformers=[('OneHotEncoder', OneHotEncoder(),
                                 Index(['TypeofContact', 'Occupation', 'Gender', 'ProductPitched',
       'MaritalStatus', 'Designation'],
      dtype='object')),
                                ('StadardScaler', StandardScaler(),
                                 Index(['Age', 'CityTier', 'DurationOfPitch', 'NumberOfFollowups',
       'PreferredPropertyStar', 'NumberOfTrips', 'Passport',
       'PitchSatisfactionScore', 'OwnCar', 'MonthlyIncome', 'TotalVisiting'],
      dtype='object'))])

In [21]:
## Applying Transformation on training dataset
xtrain = preprocessor.fit_transform(xtrain)
pd.DataFrame(xtrain)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,-0.711953,-0.779975,-0.713603,0.530883,0.434216,-0.644305,0.678514,0.784667,-0.226963,-1.496955
1,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,-0.711953,0.632573,0.293685,-0.727912,-0.681646,-0.644305,-1.518275,0.784667,0.369702,-0.080598
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,-0.711953,-0.779975,0.293685,0.530883,-0.681646,-0.644305,1.410777,0.784667,-0.265800,0.627580
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,-0.711953,-1.015400,-1.720892,-0.727912,-0.681646,-0.644305,-0.053749,0.784667,0.010892,2.043937
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,-0.711953,0.279436,1.300974,1.789679,0.434216,1.552061,1.410777,0.784667,0.973903,2.043937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3905,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,-0.711953,-0.309126,-0.713603,0.530883,-0.123715,-0.644305,-1.518275,-1.274426,-1.257601,0.627580
3906,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,-0.711953,0.985710,0.293685,-0.727912,-0.123715,1.552061,1.410777,0.784667,-0.473319,-0.080598
3907,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,-0.711953,NaN,-0.713603,1.789679,1.550078,1.552061,-0.053749,-1.274426,-0.226963,-0.788777
3908,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,-0.711953,0.044011,1.300974,1.789679,-0.681646,1.552061,-0.053749,-1.274426,-0.487424,0.627580


In [22]:
xtest = preprocessor.transform(xtest)
pd.DataFrame(xtest)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,-0.711953,-0.073701,-0.713603,-0.727912,-1.239577,-0.644305,-0.786012,0.784667,-1.109208,-1.496955
1,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,1.471370,0.044011,0.293685,0.530883,-0.681646,1.552061,1.410777,0.784667,-0.049007,-1.496955
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,-0.711953,-0.309126,-0.713603,1.789679,-1.239577,-0.644305,1.410777,0.784667,0.282753,-1.496955
3,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,-0.711953,1.103422,-0.713603,1.789679,-1.239577,1.552061,-0.053749,-1.274426,-0.586740,0.627580
4,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.471370,-0.073701,0.293685,-0.727912,2.108009,-0.644305,-0.053749,-1.274426,1.351069,2.043937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
973,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.711953,-0.191414,-0.713603,-0.727912,-0.681646,-0.644305,0.678514,0.784667,0.304200,-0.788777
974,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,-0.711953,-0.309126,1.300974,-0.727912,-0.681646,-0.644305,-0.786012,0.784667,-0.513316,1.335758
975,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.471370,-1.015400,0.293685,0.530883,-0.123715,-0.644305,0.678514,0.784667,0.473655,2.043937
976,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,1.471370,-0.073701,-0.713603,-0.727912,-0.681646,-0.644305,0.678514,0.784667,-1.142249,-1.496955


## Random Forest Classifier Training

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve

In [24]:
models = {
    'Decision Tree Classifier': DecisionTreeClassifier(),
    'Random Forest Classifier': RandomForestClassifier()
}
for i in range(len(list(models))):
  model = list(models.values())[i]
  model.fit(xtrain, ytrain)
  # make predictions
  y_train_pred = model.predict(xtrain)
  y_test_pred = model.predict(xtest)

  # Training set performance
  model_train_accuracy = accuracy_score(ytrain, y_train_pred)
  model_train_f1 = f1_score(ytrain, y_train_pred, average = 'weighted')
  model_train_recall = recall_score(ytrain, y_train_pred, average = 'weighted')
  model_train_precision = precision_score(ytrain, y_train_pred, average = 'weighted')
  model_train_roc_auc = roc_auc_score(ytrain, y_train_pred)

  # Test set performance
  model_test_accuracy = accuracy_score(ytrain, y_train_pred)
  model_test_f1 = f1_score(ytrain, y_train_pred, average = 'weighted')
  model_test_recall = recall_score(ytrain, y_train_pred, average = 'weighted')
  model_test_precision = precision_score(ytrain, y_train_pred, average = 'weighted')
  model_test_roc_auc = roc_auc_score(ytrain, y_train_pred)

  print(list(models.keys())[i])

  print('Model performance for Training set')
  print("- Accuracy: {:.4f}".format(model_train_accuracy))
  print('- F1 score: {:.4f}'.format(model_train_f1))
  print('- Recall: {:.4f}'.format(model_train_recall))
  print('- Precision: {:.4f}'.format(model_train_precision))
  print('- ROC AUC: {:.4f}'.format(model_train_roc_auc))

  print('----------------------------------')

  print('Model performance for Test set')
  print("- Accuracy: {:.4f}".format(model_test_accuracy))
  print('- F1 score: {:.4f}'.format(model_test_f1))
  print('- Recall: {:.4f}'.format(model_test_recall))
  print('- Precision: {:.4f}'.format(model_test_precision))
  print('- ROC AUC: {:.4f}'.format(model_test_roc_auc))

  print('='*35)
  print('\n')

Decision Tree Classifier
Model performance for Training set
- Accuracy: 1.0000
- F1 score: 1.0000
- Recall: 1.0000
- Precision: 1.0000
- ROC AUC: 1.0000
----------------------------------
Model performance for Test set
- Accuracy: 1.0000
- F1 score: 1.0000
- Recall: 1.0000
- Precision: 1.0000
- ROC AUC: 1.0000


Random Forest Classifier
Model performance for Training set
- Accuracy: 1.0000
- F1 score: 1.0000
- Recall: 1.0000
- Precision: 1.0000
- ROC AUC: 1.0000
----------------------------------
Model performance for Test set
- Accuracy: 1.0000
- F1 score: 1.0000
- Recall: 1.0000
- Precision: 1.0000
- ROC AUC: 1.0000




In [25]:
## Hyperparameter Training
rf_params = {'max_depth': [5,7,15,None,10],
             'max_features':[5,7,'auto',8],
             'min_samples_split':[2,8,15,20],
             'n_estimators':[100,200,500,1000]}

In [26]:
## Models list for Hyperparameter tuning
randomcv_models = [('RF', RandomForestClassifier(), rf_params)]

In [27]:
randomcv_models

[('RF',
  RandomForestClassifier(),
  {'max_depth': [5, 7, 15, None, 10],
   'max_features': [5, 7, 'auto', 8],
   'min_samples_split': [2, 8, 15, 20],
   'n_estimators': [100, 200, 500, 1000]})]

In [28]:
from sklearn.model_selection import RandomizedSearchCV
model_param = {}
for name, model, params in randomcv_models:
  random = RandomizedSearchCV(estimator=model,
                              param_distributions= params,
                              n_iter =100,
                              cv = 3,
                              verbose=2,
                              n_jobs = -1)
  random.fit(xtrain, ytrain)
  model_param[name] = random.best_params_

for model_name in model_param:
  print(f'______________________________Best Params for {model_name}___________________________')
  print(model_param[model_name])

Fitting 3 folds for each of 100 candidates, totalling 300 fits


KeyboardInterrupt: 